In [25]:
import plotly.graph_objects as go
import numpy as np
import random

In [26]:
all_obstacles = []
for i in range(30):
    x = random.randint(30, 6000)
    y = random.randint(30, 6000)
    z = 300
    r = random.randint(5, 151)
    all_obstacles.append([x, y, z, r])

In [27]:
all_obstacles

[[2384, 665, 300, 56],
 [53, 5963, 300, 140],
 [5725, 1271, 300, 124],
 [4065, 4051, 300, 22],
 [893, 5921, 300, 112],
 [1118, 5152, 300, 49],
 [2580, 3638, 300, 146],
 [1081, 2511, 300, 121],
 [2875, 4379, 300, 104],
 [2434, 2579, 300, 101],
 [4321, 5328, 300, 115],
 [698, 2551, 300, 136],
 [5454, 1667, 300, 67],
 [5562, 315, 300, 111],
 [5982, 3193, 300, 106],
 [357, 4539, 300, 113],
 [3042, 3367, 300, 40],
 [3099, 2041, 300, 126],
 [2444, 4108, 300, 64],
 [2776, 4750, 300, 40],
 [5995, 3242, 300, 44],
 [2907, 3397, 300, 91],
 [2693, 3293, 300, 13],
 [3310, 3909, 300, 37],
 [1953, 860, 300, 63],
 [5805, 5649, 300, 105],
 [162, 213, 300, 42],
 [2729, 5669, 300, 114],
 [4901, 1693, 300, 23],
 [2994, 3548, 300, 48]]

In [28]:
x = [i[0] for i in all_obstacles]
y = [i[1] for i in all_obstacles]
z = [i[2] for i in all_obstacles]
r = [i[3] for i in all_obstacles]

In [29]:
w_x,w_y,w_z = [100, 500], [100, 500], [100, 500]


In [30]:
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers', marker=dict(
        size=7,
        color=z,                  
        opacity=0.8
    ))])

fig.add_trace(
    go.Scatter3d(x=w_x, y=w_y, z=w_z, mode='markers',
        marker=dict(
        size=5)))
fig.show()

In [42]:
wingspan = 10
def distanceToObstacle(curr, goal, obstacle):
    path = [goal[i] - curr[i] for i in range(len(goal))]
    obstacle_dir = [0, 0, 1]
    reference = [goal[i] - obstacle[i] for i in range(len(goal))]
    normal = np.cross(path, obstacle_dir)
    distance = np.dot(reference, normal)/np.linalg.norm(normal)
    return abs(distance)
    

In [43]:
def dangerousObstacles(curr, goal):
    inPath = []
    for obstacle in all_obstacles:
        if distanceToObstacle(curr, goal, obstacle) < wingspan + obstacle[3]:
            inPath.append(obstacle)
            print(distanceToObstacle(curr, goal, obstacle))
    return inPath

dangerousObstacles([100, 100, 100], [500, 500, 500])

9.899494936611665
102.53048327204938
110.30865786510141
36.062445840513924


[[4065, 4051, 300, 22],
 [2434, 2579, 300, 101],
 [5805, 5649, 300, 105],
 [162, 213, 300, 42]]